* Flickr, Picasa

* Instagram is a social networking service which enables its users to upload and share their photos and videos with other users. Instagram users can choose to share information either publicly or privately. Anything shared publicly can be seen by any other user, whereas privately shared content can only be accessed by a specified set of people. Instagram also enables its users to share through many other social networking platforms, such as Facebook, Twitter, Flickr, and Tumblr.

* simpler version of Instagram, where a user can share photos and can also follow other users. The ‘News Feed’ for each user will consist of top photos of all the people the user follows.

# Requirement
* Users should be able to upload/download/view photos.
* Users can perform searches based on photo/video titles.
* Uers can follow other users.
* The system should be able to generate and display a user’s News Feed consisting of top photos from all the people the user follows.
* Like and comment on image. Does comment is recursive? Instagram only allows one layer of recursion

### Non functional
* Our service needs to be highly available.
* The acceptable latency of the system is 200ms for News Feed generation.
* Consistency can take a hit (in the interest of availability), if a user doesn’t see a photo for a while; it should be fine.
* The system should be highly reliable; any uploaded photo or video should never be lost.

### Extended
* Adding tags to photos, searching photos on tags, commenting on photos, tagging users to photos, who to follow, etc.

# Design
* The system would be read-heavy, so we will focus on building a system that can retrieve photos quickly.
* Practically, users can upload as many photos as they like. Efficient management of storage should be a crucial factor while designing this system.
* Low latency is expected while viewing photos.
* Data should be 100% reliable. If a user uploads a photo, the system will guarantee that it will never be lost.

# Capacity
* 500 M total users and 1M daily active users
* 2M new photos every day. 23 new photo every second
* Photo size 200KB
* 1 day photo space = 2M * 200KB = 400 GB
* 10 years space = 400 * 365 * 10 = 1425TB

# High level Design

* upload photos and the other to view/search photos. Our service would need some object storage servers to store photos and also some database servers to store metadata information about the photos.

![](images/instagram1.PNG)

# Database Schema

* We need to store data about users, their uploaded photos, and people they follow. Photo table will store all data related to a photo; we need to have an index on (PhotoID, CreationDate) since we need to fetch recent photos first.

![](images/instagram2.PNG)

* We can store photos in a distributed file storage like HDFS or S3.
* We can store the above schema in a distributed key-value store to enjoy the benefits offered by NoSQL. All the metadata related to photos can go to a table where the ‘key’ would be the ‘PhotoID’ and the ‘value’ would be an object containing PhotoLocation, UserLocation, CreationTimestamp, etc.

* We need to store relationships between users and photos, to know who owns which photo. We also need to store the list of people a user follows. For both of these tables, we can use a wide-column datastore like Cassandra. For the ‘UserPhoto’ table, the ‘key’ would be ‘UserID’ and the ‘value’ would be the list of ‘PhotoIDs’ the user owns, stored in different columns. We will have a similar scheme for the ‘UserFollow’ table.

* Cassandra or key-value stores in general, always maintain a certain number of replicas to offer reliability. Also, in such data stores, deletes don’t get applied instantly, data is retained for certain days (to support undeleting) before getting removed from the system permanently.

# Data size

* Assuming each “int” and “dateTime” is four bytes, each row in the User’s table will be of 68 bytes
     - UserID (4 bytes) + Name (20 bytes) + Email (32 bytes) + DateOfBirth (4 bytes) + CreationDate (4 bytes) + LastLogin (4 bytes) = 68 bytes

* If we have 500 million users, we will need 32GB of total storage.

* Photo: Each row in Photo’s table will be of 284 bytes
    - PhotoID (4 bytes) + UserID (4 bytes) + PhotoPath (256 bytes) + PhotoLatitude (4 bytes) + PhotLongitude(4 bytes) + UserLatitude (4 bytes) + UserLongitude (4 bytes) + CreationDate (4 bytes) = 284 bytes
* If 2M new photos get uploaded every day, we will need 0.5GB of storage for one day
* For 10 years we will need 1.88TB of storage.
* If we have 500 million users and on average each user follows 500 users. We would need 1.82TB of storage for the UserFollow table:
    - 500 million users * 500 followers * 8 bytes ~= 1.82TB

# Design

* Photo uploads (or writes) can be slow as they have to go to the disk, whereas reads will be faster, especially if they are being served from cache.
* Uploading users can consume all the available connections, as uploading is a slow process. This means that ‘reads’ cannot be served if the system gets busy with all the write requests. We should keep in mind that web servers have a connection limit before designing our system. If we assume that a web server can have a maximum of 500 connections at any time, then it can’t have more than 500 concurrent uploads or reads. To handle this bottleneck we can split reads and writes into separate services. We will have dedicated servers for reads and different servers for writes to ensure that uploads don’t hog the system.
* Separating photos’ read and write requests will also allow us to scale and optimize each of these operations independently.

![](images/instagram4.PNG)

# Reliability and Redundancy

* Losing files is not an option for our service. Therefore, we will store multiple copies of each file so that if one storage server dies we can retrieve the photo from the other copy present on a different storage server.
* If we want to have high availability of the system, we need to have multiple replicas of services running in the system, so that if a few services die down the system still remains available and running. Redundancy removes the single point of failure in the system.
* If only one instance of a service is required to run at any point, we can run a redundant secondary copy of the service that is not serving any traffic, but it can take control after the failover when primary has a problem.
* if there are two instances of the same service running in production and one fails or degrades, the system can failover to the healthy copy. Failover can happen automatically or require manual intervention.

![](images/instagram5.PNG)

# Data Sharding

* Let’s discuss different schemes for metadata sharding:
* Partitioning based on UserID Let’s assume we shard based on the ‘UserID’ so that we can keep all photos of a user on the same shard. If one DB shard is 1TB, we will need four shards to store 3.7TB of data. Let’s assume for better performance and scalability we keep 10 shards.

* So we’ll find the shard number by UserID % 10 and then store the data there. To uniquely identify any photo in our system, we can append shard number with each PhotoID.

* How can we generate PhotoIDs? Each DB shard can have its own auto-increment sequence for PhotoIDs and since we will append ShardID with each PhotoID, it will make it unique throughout our system.

* Problems
    - How to handle hot users? Several people follow such hot users and a lot of other people see any photo they upload.
    - Some users will have a lot of photos compared to others, thus making a non-uniform distribution of storage.
    - What if we cannot store all pictures of a user on one shard? If we distribute photos of a user onto multiple shards will it cause higher latencies?
    - Storing all photos of a user on one shard can cause issues like unavailability of all of the user’s data if that shard is down or higher latency if it is serving high load etc.

* Partitioning based on PhotoID If we can generate unique PhotoIDs first and then find a shard number through “PhotoID % 10”, the above problems will have been solved. We would not need to append ShardID with PhotoID in this case as PhotoID will itself be unique throughout the system.

* How can we generate PhotoIDs? Here we cannot have an auto-incrementing sequence in each shard to define PhotoID because we need to know PhotoID first to find the shard where it will be stored. One solution could be that we dedicate a separate database instance to generate auto-incrementing IDs. If our PhotoID can fit into 64 bits, we can define a table containing only a 64 bit ID field. So whenever we would like to add a photo in our system, we can insert a new row in this table and take that ID to be our PhotoID of the new photo.
* Wouldn’t this key generating DB be a single point of failure? Yes, it would be. A workaround for that could be defining two such databases with one generating even numbered IDs and the other odd numbered. 
* We can put a load balancer in front of both of these databases to round robin between them and to deal with downtime. Both these servers could be out of sync with one generating more keys than the other, but this will not cause any issue in our system. We can extend this design by defining separate ID tables for Users, Photo-Comments, or other objects present in our system.

* we can implement a ‘key’ generation scheme

* We can have a large number of logical partitions to accommodate future data growth, such that in the beginning, multiple logical partitions reside on a single physical database server. Since each database server can have multiple database instances on it, we can have separate databases for each logical partition on any server. So whenever we feel that a particular database server has a lot of data, we can migrate some logical partitions from it to another server. We can maintain a config file (or a separate database) that can map our logical partitions to database servers; this will enable us to move partitions around easily. Whenever we want to move a partition, we only have to update the config file to announce the change.

# Ranking and News Feed Generation

* To create the News Feed for any given user, we need to fetch the latest, most popular and relevant photos of the people the user follows.

* For simplicity, let’s assume we need to fetch top 100 photos for a user’s News Feed. Our application server will first get a list of people the user follows and then fetch metadata info of latest 100 photos from each user. In the final step, the server will submit all these photos to our ranking algorithm which will determine the top 100 photos (based on recency, likeness, etc.) and return them to the user. A possible problem with this approach would be higher latency as we have to query multiple tables and perform sorting/merging/ranking on the results. To improve the efficiency, we can pre-generate the News Feed and store it in a separate table.

* We can have dedicated servers that are continuously generating users’ News Feeds and storing them in a ‘UserNewsFeed’ table. So whenever any user needs the latest photos for their News Feed, we will simply query this table and return the results to the user.

* Whenever these servers need to generate the News Feed of a user, they will first query the UserNewsFeed table to find the last time the News Feed was generated for that user. Then, new News Feed data will be generated from that time onwards

* Pull: Clients can pull the News Feed contents from the server on a regular basis or manually whenever they need it. Possible problems with this approach are a) New data might not be shown to the users until clients issue a pull request b) Most of the time pull requests will result in an empty response if there is no new data.

*  Push: Servers can push new data to the users as soon as it is available. To efficiently manage this, users have to maintain a Long Poll request with the server for receiving the updates. A possible problem with this approach is, a user who follows a lot of people or a celebrity user who has millions of followers; in this case, the server has to push updates quite frequently.

* With long polling, the client requests information from the server exactly as in normal polling, but with the expectation the server may not respond immediately. If the server has no new information for the client when the poll is received, instead of sending an empty response, the server holds the request open and waits for response information to become available. Once it does have new information, the server immediately sends an HTTP/S response to the client, completing the open HTTP/S Request. Upon receipt of the server response, the client often immediately issues another server request.

* Hybrid: We can adopt a hybrid approach. We can move all the users who have a high number of follows to a pull-based model and only push data to those users who have a few hundred (or thousand) follows. Another approach could be that the server pushes updates to all the users not more than a certain frequency, letting users with a lot of follows/updates to regularly pull data.

# News Feed Creation with Sharded Data

* One of the most important requirement to create the News Feed for any given user is to fetch the latest photos from all people the user follows. For this, we need to have a mechanism to sort photos on their time of creation. To efficiently do this, we can make photo creation time part of the PhotoID. As we will have a primary index on PhotoID, it will be quite quick to find the latest PhotoIDs.

* We can use epoch time for this. Let’s say our PhotoID will have two parts; the first part will be representing epoch time and the second part will be an auto-incrementing sequence. So to make a new PhotoID, we can take the current epoch time and append an auto-incrementing ID from our key-generating DB. We can figure out shard number from this PhotoID ( PhotoID % 10) and store the photo there.

* What could be the size of our PhotoID? Let’s say our epoch time starts today, how many bits we would need to store the number of seconds for next 50 years?

    - 86400 sec/day * 365 (days a year) * 50 (years) => 1.6 billion seconds
* We would need 31 bits to store this number. Since on the average, we are expecting 23 new photos per second; we can allocate 9 bits to store auto incremented sequence. So every second we can store (2^9 => 512) new photos. We can reset our auto incrementing sequence every second.

# Cache and Load balancing



* Our service would need a massive-scale photo delivery system to serve the globally distributed users. Our service 
should push its content closer to the user using a large number of geographically distributed photo cache servers and use CDNs

* We can introduce a cache for metadata servers to cache hot database rows. We can use Memcache to cache the data and Application servers before hitting database can quickly check if the cache has desired rows. Least Recently Used (LRU) can be a reasonable cache eviction policy for our system.

* If we go with 80-20 rule, i.e., 20% of daily read volume for photos is generating 80% of traffic which means that certain photos are so popular that the majority of people read them. This dictates that we can try caching 20% of daily read volume of photos and metadata.

# Instagram
* SSL terminates at the ELB, which lessens the CPU load on nginx.
* Twelve PostgreSQL replicas run in a different availability zone.
* Amazon CloudFront as the CDN.
* Redis powers their main feed, activity feed, sessions system. Redis runs in a master-replica setup. Replicas constantly save to disk.
* Apache Solr powers the geo-search API
* Gearman (Gearman provides a generic application framework to farm out work to other machines or processes that are better suited to do the work. It allows you to do work in parallel, to load balance processing, and to call functions between languages. It can be used in a variety of applications, from high-availability web sites to the transport of database replication events. In other words, it is the nervous system for how distributed processing communicates.) is used to: asynchronously share photos to Twitter, Facebook, etc; notifying real-time subscribers of a new photo posted; feed fan-out. 200 Python workers consume tasks off the Gearman task queue.

* Everyday :
    - 400 Million users visits
    - 4 Billion likes
    - 100 million photo/video upload
    - 110 million followers for top account

* Instagram stack
* Web tier in python Django, receive request and access other services for response.
* Some user request are handled asynchronously in backend (Liking someone photo and sending notification to the person), push it to queue like RebbitMQ
* Storage vs Computing
    - Storage needs to be consistent across data centers with replication (perhaps with latency but eventual consistent)
    - Computing driven by user traffic as needed basis (Stateless, data in servers are temporary and can reconstructed from global data)
* PostgresSQL : User, media, friendship etc.
    - Deployed in every region as master and multiple replica.
    - Django write to masters of multiple region, but read is conducted only from local region.
![](images/instagram7.png)

* Cassandra:
    - User feeds, activities
    - There is no master, all replica has same copy of data with eventual consistency
    - Consistency can be configured as per application tolerance level, some can set to write consistency of 2, and read to 1.
    - We can have repica living in different data centers
![](images/instagram8.PNG)

* For computing
    - Combine Django, RabbitMQ and celery (Celery is an open source asynchronous task queue or job queue which is based on distributed message passing.) in one pod that goes in each region, global load balancer will balance the user request to Django. Asynchronous task is produced and consume in the same region.
![](images/instagram9.png)

* In image, database replication across the region and computing resources contained within the region

* Memcache
    - High performance key-val store in memory
    - Millions of reads/write per second
    - Sensitive to network condition because of large number of request and so latency can be affected, so cross region read/write is prohibited.
    - No global consistent memcache
    - Memcache in each region is determined by user traffic served out of that region.

![](images/instagram10.png)

* In same region, user C make comment saved in both db and cache, in same region user R wants feed , comment can be read from memcache.

![](images/instagram11.png)
* Other user connected to different DC, DC2
* From DC1 comment gets replicated to db of DC2. But cache of DC2 is still the stale data.

* Instead Django update a cache, make db to update cache
![](images/instagram12.PNG)

* It can create a load on postgres server

* There is a one image with 1.2 million likes

```
SELECT COUNT(*) form user_likes_media WHERE media_id = 1234;
```
* Takes 100 ms
* Before when we used to write in cache directly it was the issue of incrementing a counter in memcache which is very fast
* But now after every like postgres invalidate the cache and calculate new likes and update the cache.
* Denormalize the db, which stores mediaid and number of likes,
```
SELECT count from media_like where media_id = 1234;
```
* When cache is invalidated all the read likes request go to db, which increases the traffic

* Memcache lease
![](images/instagram13.PNG)

* First django server try to read from memcache as lease-get, memcache will permit it to go to db, meanwhile if second request comes to memcache, it will not allow to go to db, but suggested to use stale data. After first server fetch the value from db, it will do lease-set to memcache, after that it updates the cache. 
* So, leases work in practice because only one client is headed to the DB at any given time for a particular value in the cache.

* when a new delete comes in, memcached will know that the next lease-set (the one from client A) is already stale, so it will accept the value (because it's newer) but it will mark it as stale and the next client to ask for that value will get a new lease token, and clients after that will once again get hot miss results.

* Use few CPU instructions as possible
    - Writing good code
* Use few server as possible to handle request

* Use profiling tool to see which part of code is taking high time.

* When client request feed, server return the URL of CDN where client can find the media.
* URL can be generated using URL generator,
* URL is generated depends on where the user is located, what kind of network it is connected with, what kind of device is there
    - To generae different URL for different media size they used to call URL generator multiple times.
    - Instead call once and overwrite the size function.
* Function which are stable and used extensively develop them in C/C++

* To get more from single server, run multiple instance of worker process in the server. But it is upper bounded by server memory. There is private memory for each worker and there is a shared memory.
* How to reduce memory requirement to run more processes
    - Reduce the code in memory
    - Remove dead code (Cprofile)
    - Provide restrictive private memory
    - move configuration into shared memory (treadoff of latency)
    - Disable garbage collection (Python requires GC in private part only)
* Each process can handle one request at a time so while it is waiting for external service wastage of resources.
    - Home feed has stories, posts. Which can increase latency for retrieval.
    - Instead of requesting suequentially to feed, stories and suggested users services, we can have asynchronous IO to access simultaneously.

* Which server?
* New table or new column?
* What index?
* Should i cache it?
* Will I lock up DB?

* Tao
    - Database plus write through cache, Used RDBMS as a backend to store data, but very simplified model. Uss nodes as an object and edges with relationship.
    - We can not make direct sophosticated queries in db itself.
![](images/instagram14.PNG)

* Load test:
    - Artificial load, that are semi triggered by user request. how many users will be using that features

* Every request to Instagram servers goes through load balancing machines; we used to run 2 nginx machines and DNS Round-Robin between them. The downside of this approach is the time it takes for DNS to update in case one of the machines needs to get decomissioned. Recently, we moved to using Amazon’s Elastic Load Balancer, with 3 NGINX instances behind it that can be swapped in and out (and are automatically taken out of rotation if they fail a health check). We also terminate our SSL at the ELB level, which lessens the CPU load on nginx. 

* The photos themselves go straight to Amazon S3, which currently stores several terabytes of photo data for us. We use Amazon CloudFront as our CDN, which helps with image load times from users around the world

* When a user decides to share out an Instagram photo to Twitter or Facebook, or when we need to notify one of our Real-time subscribers of a new photo posted, we push that task into Gearman, a task queue system originally written at Danga. Doing it asynchronously through the task queue means that media uploads can finish quickly, while the ‘heavy lifting’ can run in the background. We have about 200 workers (all written in Python) consuming the task queue at any given time, split between the services we share to. We also do our feed fan-out in Gearman, so posting is as responsive for a new user as it is for a user with many followers.

* how to assign unique identifiers to each piece of data in the database (for example, each photo posted in our system). The typical solution that works for a single database — just using a database’s natural auto-incrementing primary key feature — no longer works when data is being inserted into many databases at the same time. 

* Generated IDs should be sortable by time
* IDs should ideally be 64 bits (for smaller indexes, and better storage in systems like Redis
* Generate IDs in web application
    - This approach leaves ID generation entirely up to your application, and not up to the database at all. For example, MongoDB’s ObjectId, which is 12 bytes long and encodes the timestamp as the first component. Another popular approach is to use UUIDs.
    - Each application thread generates IDs independently, minimizing points of failure and contention for ID generation
    - If you use a timestamp as the first component of the ID, the IDs remain time-sortable
    - Generally requires more storage space (96 bits or higher) to make reasonable uniqueness guarantees
    - Some UUID types are completely random and have no natural sort
* Generate IDs through dedicated service
    - Twitter’s Snowflake, a Thrift service that uses Apache ZooKeeper to coordinate nodes and then generates 64-bit unique IDs
    - Snowflake IDs are 64-bits, half the size of a UUID
    - Can use time as first component and remain sortable
    - Distributed system that can survive nodes dying
    - Would introduce additional complexity and more ‘moving parts’ (ZooKeeper, Snowflake servers) into our architecture
* DB Ticket Servers
    - Uses the database’s auto-incrementing abilities to enforce uniqueness. Flickr uses this approach, but with two ticket DBs (one on odd numbers, the other on even) to avoid a single point of failure.
    - 

# Flicker

* Statelessness means they can bounce people around servers and it's easier to make their APIs.
* Scaled at first by replication, but that only helps with reads.
* Create a search farm by replicating the portion of the database they want to search.
* - Shards: My data gets stored on my shard, but the record of performing action on your comment, is on your shard. When making a comment on someone else's’ blog

* Clicking a Favorite:
    - Pulls the Photo owners Account from Cache, to get the shard location (say on shard-5)
    - Pulls my Information from cache, to get my shard location (say on shard-13)
    - Starts a “distributed transaction” - to answer the question: Who favorited the photo? What are my favorites?

* Each server in shard is 50% loaded. Shut down 1/2 the servers in each shard. So 1 server in the shard can take the full load if a server of that shard is down or in maintenance mode. To upgrade you just have to shut down half the shard, upgrade that half, and then repeat the process.

* Average queries per page, are 27-35 SQL statements. Favorites counts are real time. API access to the database is all real time.

- A lot of data is stored twice. For example, a comment is part of the relation between the commentor and the commentee. Where is the comment stored? How about both places? Transactions are used to prevent out of sync data: open transaction 1, write commands, open transaction 2, write commands, commit 1st transaction if all is well, commit 2nd transaction if 1st committed. but there still a chance for failure when a box goes down during the 1st commit.

* Photos are stored on the filer. Upon upload, it processes the photos, gives you different sizes, then its complete. Metadata and points to the filers, are stored in the database.

* Tags do not fit well with traditional normalized RDBMs schema design. Denormalization or heavy caching is the only way to generate a tag cloud in milliseconds for hundreds of millions of tags.

* Some of their data views are calculated offline by dedicated processing clusters which save the results into MySQL because some relationships are so complicated to calculate it would absorb all the database CPU cycles.

A typical URL for a Flickr image looks like this:

http://farm1.static.flickr.com/104/301293250_dc284905d0_m.jpg

If we split this up we get:

farm1 - Obviously the farm at which the image is stored. I have yet to see a value other than one.

.static.flickr.com - Fairly self explanitory.

/104 - The server ID number.

/301293250 - The image ID.

_dc284905d0 - The image 'secret'. I assume this is to prevent images being copied without first getting the information from the API.

_m - The size of the image. In this case the 'm' denotes medium, but this can be small, thumb etc. For the standard image size there is no size of this form in the URL.

![](images/instagram3.PNG)